# 전처리
※ 주의사항: 전처리 관련 모든 파일은 현재 폴더에 저장해주세요

In [1]:
import pandas as pd
import numpy as np 
import re

In [2]:
pro_df = pd.read_csv('./크롤링 프로그램/programmers.csv')  # 기존
dart_df = pd.read_csv('./크롤링 프로그램/dart_result.csv', index_col=0)  # 다트
want_df = pd.read_csv('./크롤링 프로그램/wantedinsight_result.csv', index_col= 0)  # 원티드 인사이트
incru_df = pd.read_csv('./크롤링 프로그램/incruit_result.csv')  # 인크루트
cat_df = pd.read_csv('./크롤링 프로그램/catch_result.csv', index_col=0)  # 캐치

# 1. 프로그래머스 전처리
- 주소: 시(도), 구
- 사원수: 숫자만, int타입
- 직급: 최소단위 기준, int타입

In [3]:
# 주소 전처리
address_list = []

for ad in pro_df['주소']:
    # 우편번호 패턴을 찾아서 제거
    ad = re.sub('^\d{5}\s', '', ad).strip()
    # 글자 변경
    if '대한민국' in ad:
        ad = re.sub('대한민국', '', ad).strip()
    elif '서울' in ad and '특별시' not in ad:
        ad = re.sub('서울','서울특별시',ad).strip()
    elif '경기' in ad and '경기도' not in ad:
        ad = re.sub('경기','경기도',ad).strip()
    elif '인천' in ad and '광역시' not in ad:
        ad = re.sub('인천','인천광역시',ad).strip()

    address_list.append(ad)

# 구,시,도로 끝나는 단어 리스트 정렬
add_list = [' '.join(re.findall(r'\b(\w+[구시도])\b', address)) for address in address_list]

pro_df['주소'] = add_list

In [4]:
# 사원수 전처리
employee_list = []

# '명' 및 공백 제거
for emp in pro_df['사원수']:
    employee_list.append(int(re.sub('명','',emp).strip()))

pro_df['사원수'] = employee_list
pro_df['사원수'] = pro_df['사원수'].astype(int)

In [5]:
# 직급 전처리
career_list = []

for car in pro_df['직급']:
    if '무관' not in car:  # 무관이 없는 경우 최소단위
        career_list.append(car.split('~')[0].strip())

    elif '신입' in car:  # 신입인 경우 0
        career_list.append(0)
                           
    else:  # 무관이 있는 경우 0
        career_list.append(0) 

pro_df['직급'] = career_list
pro_df['직급'] = pro_df['직급'].astype(int)  # int타입 변경

In [6]:
pro_df.to_csv('./pro_pre.csv')  # 전처리된 파일 저장

# 2. 다트 전처리
- 설립연도
- 주소

In [7]:
dart_df.설립연도 = dart_df.설립연도.apply(lambda x: ''.join(re.findall('\d\d\d\d', x)))  # 설립연도
dart_df.주소 = dart_df.주소.apply(lambda x: ' '.join(re.findall(r'\b(\w+[구시도])\b', x)))  # 주소
dart_df.설립연도 = dart_df.설립연도.astype(int)  # 타입변경

dart_df.reset_index(drop=True, inplace=True)  # 인덱스 재설정

In [8]:
dart_df.to_csv('./dart_pre.csv')

# 3. 원티드 인사이트 전처리
- 업종분류
- 설립연도
- 매출액
- 사원수

In [9]:
# 잘못된 값 결측치로 변경
want_df.loc[want_df.업종분류 == '-', '업종분류'] = None  # 업종분류
want_df.loc[want_df.매출액 == '-원', '매출액'] = None  # 매출액

In [10]:
# 설립연도
want_df.설립연도 = want_df.설립연도.apply(lambda x: int(''.join(re.findall('(\d\d\d\d)', x))))

In [11]:
# 사원수 잘못된 값 결측치로 변경
want_df.loc[want_df.사원수 == '정보없음', '사원수'] = None
want_df.loc[want_df.사원수 == 'nan', '사원수'] = None

In [12]:
# 매출액
want_df_sales = []
want_df_sales_res = []

for q in want_df['매출액']:
    if q != None:
        want_df_sales.append(q.replace(',',''))
    else:
        want_df_sales.append(None)

try:
    for q in want_df_sales:
        if q != None: 
            if '조' in q:
                    want_df_sales_res.append(int(''.join(re.findall(r'(\d+)조', q)))*100000000 + int(''.join(re.findall(r'(\d+)억', q)))*10000 + int(''.join(re.findall(r'억(\d+)', q))))
            elif '억원' in q:
                 want_df_sales_res.append(int(''.join(re.findall(r'(\d+)억원', q)))*10000)
            elif '억' in q:
               want_df_sales_res.append(int(''.join(re.findall(r'(\d+)억', q)))*10000 + int(''.join(re.findall(r'억(\d+)', q))))   
            else:
                want_df_sales_res.append(int(''.join(re.findall(r'(\d+)만원', q))))
        else:
            want_df_sales_res.append(None)
except:
     print(q)

want_df['매출액'] = want_df_sales_res

In [13]:
# 사원수
want_df_emp = []

for f in want_df['사원수']:
    if pd.notna(f):
        want_df_emp.append(int(''.join(re.findall('\d', f))))
    else:
        want_df_emp.append(None)
        
want_df.사원수 = want_df_emp

# 인덱스 재설정
want_df.reset_index(drop=True, inplace=True)

In [14]:
want_df.to_csv('./want_pre.csv')  # 전처리된 파일 저장

# 4. 인크루트 전처리

In [15]:
# 설립연도
corp_year_df = []

for ss in incru_df.설립연도:
    if ss != '-':
        corp_year_df.append(int(''.join(re.findall('\d\d\d\d', ss))))
    else:
        corp_year_df.append(None)


incru_df.설립연도 = corp_year_df

In [16]:
# 매출액
sales_data = []
sales_df = []
data_sales_res = []

for sales in incru_df.매출액:
    if sales != '-':
        sales_data.append(sales.replace(',',''))
    else:
        sales_data.append(None)

for qq in sales_data:
    if qq != None:
        sales_df.append(re.sub(r'\(\d{4}\)', '', qq.replace(' ','')))
    else:
        sales_df.append(None)

for q in sales_df:
    if q != None: 
        if '조' in q:
                data_sales_res.append(int(''.join(re.findall(r'(\d+)조', q)))*100000000 + int(''.join(re.findall(r'(\d+)억', q)))*10000 + int(''.join(re.findall(r'억(\d+)', q))))
        elif '억원' in q:
                data_sales_res.append(int(''.join(re.findall(r'(\d+)억원', q)))*10000)
        elif '억' in q:
            data_sales_res.append(int(''.join(re.findall(r'(\d+)억', q)))*10000 + int(''.join(re.findall(r'억(\d+)', q))))   
        else:
            data_sales_res.append(int(''.join(re.findall(r'(\d+)만원', q))))
    else:
        data_sales_res.append(None)

incru_df.매출액 = data_sales_res

In [17]:
# 사원수
emp_df = []

for ww in incru_df.사원수:
    if ww != '-':
        emp_df.append(int(''.join(re.findall('\d', ww))))
    else:
        emp_df.append(None)

incru_df.사원수 = emp_df

In [18]:
# 주소
incru_df.주소.fillna(2)  # 결측치 채우기

address_list = []
add_list = []

for ee in incru_df.주소:
    try:    
        if '경기' in ee and '경기도' not in ee:
            ee = re.sub('경기', '경기도', ee)
        elif '서울' in ee and '서울특별시' not in ee:
             ee = re.sub('서울', '서울특별시', ee)
        elif '강남구' in ee and '서울특별시' not in ee and '서울' not in ee:
            ee = re.sub('강남구', '서울특별시 강남구', ee)
    except:
        ee = 0
        
    address_list.append(ee)

for tt in address_list:
    if tt != 0:
        add_list.append(' '.join(re.findall(r'\b(\w+[구시도])\b', tt)))
    else:
        add_list.append(None)

incru_df.주소 = add_list

In [19]:
# 기업 홈페이지 URL
incru_df['기업 홈페이지 URL'] = incru_df['기업 홈페이지 URL'].apply(lambda x: str(x)[2:])
incru_df.loc[incru_df['기업 홈페이지 URL'] == 'n', '기업 홈페이지 URL'] = None

# 인덱스 재설정
cat_df.reset_index(drop=True, inplace=True)

# 5. 캐치 전처리
- 매출액: 만의 단위로, int타입
- 사원수: 숫자만, int타입
- 설립연도: 연도만, int타입

In [20]:
# 매출액
pay_list = []

for pay in cat_df['매출액']:
    # 결측치 만들기
    if '-' in pay:
        pay_list.append(None)    
    else:
        pay_list.append(int(re.findall('\d+',pay)[0])*10000)
    # 

cat_df['매출액'] = pay_list

In [21]:
# 사원수
peo_list =[]

for p in cat_df['사원수']:
    if '-명' in p:
        peo_list.append(None)
    elif '명' in p:
        peo_list.append(int(re.findall('\d+',p)[0]))
    else:
        peo_list.append(None)

cat_df['사원수'] = peo_list

In [22]:
# 설립연도
year_list = []

for y in cat_df['설립연도']:
    year_list.append(int(re.findall('\d+', y)[0]))

cat_df['설립연도'] = year_list

# 인덱스 재설정
cat_df.reset_index(drop=True, inplace=True)

In [23]:
cat_df.to_csv('./cat_pre.csv')  # 전처리된 파일 저장

# 기존 데이터에 전처리된 파일들 머지하기
- 머지 순서는 위에서 정리한 1번부터 5번까지 진행됩니다.

In [25]:
df_merge = pro_df.combine_first(dart_df).combine_first(want_df).combine_first(incru_df).combine_first(cat_df)
df_merge.to_csv('./programmers_end.csv')
df_merge.to_excel('./전처리 완료.xlsx')